In [70]:
from bs4 import BeautifulSoup
import requests

url = "http://web.archive.org/web/20110514112442/http://unstats.un.org/unsd/demographic/products/socind/education.htm"

r = requests.get(url)

In [71]:
import pandas as pd
soup = BeautifulSoup(r.content)
soup = soup('table')[6]
df = pd.DataFrame(columns=['Country','Year','Total','Men','Women'])
j = 0
for i in soup.findAll('tr')[8:-11]:
    col = i.findAll('td')
    #print col[0].text
    #print col[1].text
    #print col[4].text
    #print col[7].text
    #print col[10].text
    df.loc[j] = [col[0].text,col[1].text,col[4].text,col[7].text,col[10].text]
    j += 1    

In [72]:
df.set_index('Country',inplace=True)

In [73]:
df

,Year,Total,Men,Women
Country,,,,
Afghanistan,2004,8,11,5
Albania,2004,11,11,11
Algeria,2005,13,13,13
Andorra,2008,11,11,12
Anguilla,2008,11,11,11
Antigua and Barbuda,2000,14,12,16
Argentina,2007,16,15,17
Armenia,2009,12,12,13
Aruba,2009,13,13,13


In [74]:
import numpy as np
df['Men'] = df['Men'].astype(np.int64)
df['Women'] = df['Women'].astype(np.int64)
df['Total'] = df['Total'].astype(np.int64)

In [75]:
import sqlite3 as lite

con = lite.connect('education.db')
cur = con.cursor()

df.reset_index()

df.to_sql(con=con, name='school_life', if_exists='replace', flavor='sqlite')

In [87]:
import csv

con = lite.connect('gdp.db')
cur = con.cursor()

with open('37471cbd-1b7d-43a0-ba94-e55c3c7fc0a5_v2.csv','rU') as inputFile:
    next(inputFile) # skip the first 3 lines
    next(inputFile)
    next(inputFile)
    header = next(inputFile)
    inputReader = csv.reader(inputFile)
    columns = next(inputReader)[43:-5]  
    columns = ["country_name"] + ["_" + year for year in columns]
    inputs = ["country_name TEXT"] + [column + " NUMERIC" for column in columns[1:]]
    cur.execute("DROP TABLE IF EXISTS gdp")
    cur.execute("CREATE TABLE gdp " + "(" + ", ".join(inputs) + ")")
    for line in inputReader:
        with con:
            country = line[0]
            gdp_vals = line[43:-5]
            
            cur.execute("INSERT INTO gdp " + "(" + ", ".join(columns) + ") " +
                        'VALUES ("' + country + '","' + '","'.join(gdp_vals) + '");')

In [88]:
gdp_df = pd.read_sql_query("SELECT * FROM gdp ORDER BY country_name", con, index_col = "country_name")
gdp_df.head()

,_1999,_2000,_2001,_2002,_2003,_2004,_2005,_2006,_2007,_2008,_2009,_2010,_2011
country_name,,,,,,,,,,,,,
Afghanistan,,,2.46167e+09,4.12882e+09,4.58365e+09,5.28546e+09,6.27508e+09,7.0576e+09,9.84384e+09,1.01905e+10,1.24869e+10,1.59368e+10,1.79302e+10
Albania,3.41476e+09,3.63204e+09,4.06076e+09,4.43508e+09,5.74695e+09,7.31487e+09,8.15855e+09,8.99264e+09,1.0701e+10,1.28814e+10,1.20442e+10,1.1927e+10,1.28909e+10
Algeria,4.86406e+10,5.47901e+10,5.47447e+10,5.67603e+10,6.78638e+10,8.5325e+10,1.03198e+11,1.17027e+11,1.34977e+11,1.71001e+11,1.37211e+11,1.61207e+11,2.00013e+11
American Samoa,,,,,,,,,,,,,
Andorra,1.23984e+09,1.40169e+09,1.484e+09,1.71756e+09,2.37384e+09,2.91691e+09,3.24813e+09,3.53645e+09,4.01079e+09,4.00135e+09,3.64986e+09,3.34632e+09,3.42724e+09
